# Python Inheritance

## 1. The Concept of Inheritance

Inheritance is a fundamental principle of Object-Oriented Programming (OOP) that allows a new class (Child) to borrow features (properties and methods) from an existing class (Parent).

* **Reusability:** You write the code once in the parent class and reuse it in multiple child classes.
* **Extensibility:** You can add new features to the child class without modifying the parent class.

### Engineering Analogy: The Television

* **Parent Class (Old TV):** Has basic features like `turn_on()`, `change_channel()`, and `volume_up()`.
* **Child Class (Smart TV):** Inherits all basic features from the Old TV but adds new ones like `connect_to_wifi()` and `stream_netflix()`. You don't need to reinvent the "volume" logic; you just borrow it.

---

## 2. Syntax: Creating a Child Class

To inherit from another class in Python, you simply pass the Parent Class name in parentheses when defining the Child Class.

```python
class ParentClass:
    pass

class ChildClass(ParentClass):
    pass

```

### The Scenario: Rectangle to Cuboid

A **Rectangle** has length and breadth. A **Cuboid** is just a rectangle extended into 3D space with an added height. Therefore, `Cuboid` should inherit from `Rectangle`.

---

## 3. The "Missing Link": `super()`

The transcript highlights a critical error: **AttributeError: 'Cuboid' object has no attribute 'length'**.

### Why did this happen?

When you define a custom `__init__` method in the Child class (`Cuboid`), it **overrides** (replaces) the Parent's `__init__` method. The Parent's initialization code never runs, so `self.length` and `self.breadth` are never created!

### The Solution

You must explicitly call the Parent's constructor using the `super()` function.

```python
class Rectangle:
    def __init__(self, length, breadth):
        self.length = length
        self.breadth = breadth

    def area(self):
        return self.length * self.breadth

class Cuboid(Rectangle):
    def __init__(self, length, breadth, height):
        # 1. Call the Parent's constructor using super()
        # This initializes 'length' and 'breadth' inside the Rectangle logic
        super().__init__(length, breadth)
        
        # 2. Initialize the new property specific to Cuboid
        self.height = height

    def volume(self):
        # We can access 'length' and 'breadth' because super().__init__ created them
        return self.length * self.breadth * self.height

# --- Usage ---
c = Cuboid(10, 5, 3)

# Access Parent's method
print(f"Base Area: {c.area()}")   # Output: 50

# Access Child's method
print(f"Volume: {c.volume()}")    # Output: 150

```

### Key Takeaways

1. **Inheritance Syntax:** `class Child(Parent):`
2. **Overriding `__init__`:** If you write an `__init__` in the child, you must call `super().__init__()` to ensure the parent initializes its part of the object.
3. **`super()`:** A built-in function that returns a temporary object of the parent class, allowing you to call its methods.

# Constructors in Inheritance (`super()`)

## Overview

In the previous module, we established that a Child class inherits attributes and methods from a Parent class. However, a critical issue arises when the Child class defines its own `__init__` method.

In Python, **defining a method in the Child class with the same name as the Parent class "overrides" it.**

This means if `Cuboid` defines `__init__`, the `Rectangle`'s `__init__` is **ignored**. Consequently, the `length` and `breadth` attributes are never initialized, leading to an `AttributeError`. To fix this, we must explicitly chain the constructors using `super()`.

---

## 1. The Problem: Constructor Overriding

Unlike languages like Java or C++, Python **does not** automatically call the Parent's constructor if the Child overrides it.

### The Broken Code

```python
class Rectangle:
    def __init__(self, length, breadth):
        self.length = length
        self.breadth = breadth

class Cuboid(Rectangle):
    def __init__(self, height):
        # ⚠️ PROBLEM: This completely replaces Rectangle's __init__
        # self.length and self.breadth are NEVER created!
        self.height = height

# c = Cuboid(10) # Crashes when accessing c.length

```

---

## 2. The Solution: The `super()` Function

The `super()` function provides a reference to the Parent class (Superclass). It allows the Child to call methods from the Parent that it has overridden.

By calling `super().__init__()`, we force the Parent to run its initialization logic, ensuring all inherited attributes are created correctly.

### The Logic Flow

1. **Input:** Pass all required arguments (`length`, `breadth`, `height`) to the Child.
2. **Delegate:** The Child passes the relevant arguments (`length`, `breadth`) up to the Parent via `super()`.
3. **Initialize Local:** The Child initializes its specific arguments (`height`).

---

## 3. Engineering Implementation

Here is the corrected code for the `Cuboid` class inheriting from `Rectangle`.

```python
class Rectangle:
    def __init__(self, length, breadth):
        print(f"Rectangle Init: Initializing {length}x{breadth}")
        self.length = length
        self.breadth = breadth

    def area(self):
        return self.length * self.breadth

class Cuboid(Rectangle):
    def __init__(self, length, breadth, height):
        # 1. Delegate basic initialization to the Parent
        # We pass 'length' and 'breadth' up the chain
        super().__init__(length, breadth)
        
        # 2. Initialize the specific property for this class
        self.height = height
        print(f"Cuboid Init: Initializing height {height}")

    def volume(self):
        # We can access length/breadth because super() created them
        return self.length * self.breadth * self.height

# --- Usage ---

# Creating the object triggers the chain of constructors
c = Cuboid(10, 5, 3)
# Output:
# Rectangle Init: Initializing 10x5
# Cuboid Init: Initializing height 3

print(f"Volume: {c.volume()}") # Output: 150

```

### Why not just set `self.length = length` in Cuboid?

You *could* technically write `self.length = length` inside `Cuboid` and it would work. However, this is **Bad Practice (Anti-Pattern)** because:

1. **Code Duplication:** You are rewriting logic that already exists in `Rectangle`.
2. **Maintenance Nightmare:** If `Rectangle` changes its logic (e.g., adds validation to `length`), `Cuboid` won't get that update automatically.
3. **Violation of Inheritance:** The point of inheritance is to *reuse* the parent's logic, not copy-paste it.

---

## Summary

* **Overriding:** Defining `__init__` in a Child class stops the Parent `__init__` from running.
* **`super()`:** The standard tool to access Parent methods.
* **Best Practice:** Always call `super().__init__(args)` at the start of your Child's constructor to ensure the base state is set up correctly.

# Data Hiding in Python (Encapsulation)

## Overview

In many object-oriented languages (like Java or C++), data hiding is enforced via strict keywords: `public`, `private`, and `protected`.

Python does **not** use keywords for this. Instead, it relies on **Naming Conventions** (specifically, the use of underscores `_`) to dictate how data should be accessed. This aligns with Python's philosophy of "We are all consenting adults here"—it signals intent rather than enforcing strict barriers.

---

## 1. Public Members

**Syntax:** No underscores (e.g., `self.data`).

These are accessible from anywhere: inside the class, inside child classes, and outside the class (via the object). This is the default behavior in Python.

```python
class Parent:
    def __init__(self):
        # Public: Accessible everywhere
        self.data = 10 

    def show(self):
        print(f"Inside Class: {self.data}")

# --- Usage ---
obj = Parent()
obj.show()           # Output: Inside Class: 10

# Accessible Outside
print(obj.data)      # Output: 10

# Modifiable Outside
obj.data = 20
print(obj.data)      # Output: 20

```

---

## 2. Protected Members

**Syntax:** Single Underscore (e.g., `self._data`).

**The Convention:** A single underscore indicates to other developers: *"This is an internal variable. Please do not access or modify it directly from outside the class, although you physically can."*

It behaves exactly like a public member (no error is raised if you access it), but standard engineering practice dictates you should treat it as private. It is fully accessible to **Child Classes**.

```python
class Parent:
    def __init__(self):
        # Protected: "Internal use only" convention
        self._data = 10 

class Child(Parent):
    def display(self):
        # Accessible in Child class
        print(f"Child accessing protected: {self._data}")

# --- Usage ---
c = Child()
c.display()          # Output: Child accessing protected: 10

# Technically accessible outside (BUT BAD PRACTICE)
print(c._data)       # Output: 10

```

---

## 3. Private Members

**Syntax:** Double Underscore (e.g., `self.__data`).

**The Barrier:** This provides the strongest form of data hiding in Python. Python employs a technique called **Name Mangling** to make these variables difficult (though not impossible) to access from outside or even from Child classes.

### How Name Mangling Works

When you write `self.__data`, Python internally renames the variable to `_ClassName__data`.

1. **Outside Access:** `obj.__data` raises an `AttributeError`.
2. **Child Class Access:** The child class cannot access `self.__data` because it tries to look for `_Child__data`, but the variable exists as `_Parent__data`.

```python
class Parent:
    def __init__(self):
        # Private: Name Mangling applies
        self.__data = 10 

    def show(self):
        # Accessible inside the SAME class
        print(f"Parent Data: {self.__data}")

class Child(Parent):
    def display(self):
        # NOT Accessible in Child class
        # This will raise AttributeError
        print(self.__data) 

# --- Usage ---
p = Parent()
p.show()  # Output: Parent Data: 10

# 1. Direct Access Fails
# print(p.__data) # Error: 'Parent' object has no attribute '__data'

# 2. Accessing via "Name Mangling" (The Backdoor)
# Syntax: _ClassName__VariableName
print(p._Parent__data) # Output: 10 (It works, but don't do this!)

```

---

## Summary Table

| Access Specifier | Syntax | Accessible Inside Class? | Accessible in Child Class? | Accessible Outside (Object)? |
| --- | --- | --- | --- | --- |
| **Public** | `self.name` | ✅ Yes | ✅ Yes | ✅ Yes |
| **Protected** | `self._name` | ✅ Yes | ✅ Yes | ✅ Yes (Convention: No) |
| **Private** | `self.__name` | ✅ Yes | ❌ No | ❌ No (Unless Mangled) |

### Engineering Recommendation

* **Default:** Use **Public** (`self.name`) for standard attributes.
* **Internal Logic:** Use **Protected** (`self._name`) for internal helper variables or methods that users shouldn't touch.
* **Strict Hiding:** Use **Private** (`self.__name`) *only* when you need to avoid naming conflicts with subclasses (e.g., in complex frameworks). Overusing double underscores makes debugging difficult.